# * Sales Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  DIM_CORP_KPI

In [2]:
''' Execute Summary Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summary DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-21, 10:34:28

   -> Summary DataFrame : 1054 rows, 18 columns

TDMDBPR : Disconnected


In [3]:
def my_metric_group(v_name):

    # Get : Parameter
    name = v_name
    result = ''
    
    if re.search('Gross Adds', name): result = 'Gross Adds'
    elif re.search('Inflow M1', name): result = 'Inflow M1'
    elif re.search('Inflow M2', name): result = 'Inflow M2'
    elif re.search('No of AP 1D', name): result = 'No of AP 1D'
    elif re.search('Amt of AP 1D', name): result = 'Amt of AP 1D'
    elif re.search('No of AP In Month', name): result = 'No of AP In Month'
    elif re.search('Amt of AP In Month', name): result = 'Amt of AP In Month'
    # elif re.search('CConnected : onsumer', name): result = 'Gross Adds - Con - Consumer'
    # elif re.search('Connected : DataService', name): result = 'Gross Adds - Con - DataService'
    # elif re.search('Registration : Consumer', name): result = 'Gross Adds - Reg - Consumer'
    # elif re.search('Registration : DataService', name): result = 'Gross Adds - Reg - DataService'
    else: result = 'Unknown' 
    
    return result

In [4]:
''' Add Columns '''

chk_src_df['CHANNEL_GRP'] = np.where(chk_src_df['METRIC_CD'].str.contains('CS$'), 'ALL', 'GEO')
chk_src_df['LOCATION_GRP'] = np.where(chk_src_df['METRIC_NAME'].str.contains('Location'), 'Customer', 'Sales')
chk_src_df['METRIC_GRP'] = chk_src_df.apply(lambda x: my_metric_group(str(x['METRIC_NAME'])), axis=1)
# tmp_df['MAIN_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')) | (tmp_df['METRIC_NAME'].str.contains('New Revenue|Existing Revenue')), 'N', 'Y')
# chk_src_df.tail()

# chk_src_df.loc[chk_src_df['PRODUCT']=='TOL'].tail()

### KPIs Summary

In [5]:
''' PREPAID Summary : 14 rows, 24 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'PREPAID'
# channel_grp = 'GEO'

prepaid_df = chk_src_df.copy()
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==curr_mth]
prepaid_df = prepaid_df.loc[prepaid_df['PRODUCT']==product]
# prepaid_df = prepaid_df.loc[prepaid_df['CHANNEL_GRP']==channel_grp]

prepaid_df['ALL'] = prepaid_df['P']
prepaid_df['TRUE'] = np.where(prepaid_df['COMP_CD']=='TRUE', prepaid_df['P'], 0)
prepaid_df['DTAC'] = np.where(prepaid_df['COMP_CD']=='DTAC', prepaid_df['P'], 0)
prepaid_df['TMP_NAME'] = prepaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

prepaid_df = prepaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
prepaid_df = prepaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = prepaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    prepaid_df[col] = prepaid_df[col].apply(lambda x: format(x, ',.0f'))

prepaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D,2025-03-21 08:01:03,20250301,20250320,"243,559,378","119,413,652","124,145,726"
1,PREPAID,Amt of AP 1D,Prepaid Amt of AP 1D - GEO Channel,2025-03-21 08:01:03,20250301,20250320,"238,695,182","118,202,759","120,492,423"
2,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month,2025-03-21 08:01:03,20250301,20250320,"259,235,981","125,714,837","133,521,144"
3,PREPAID,Amt of AP In Month,Prepaid Amt of AP In Month - GEO Channel,2025-03-21 08:01:03,20250301,20250320,"254,143,898","124,489,538","129,654,360"
4,PREPAID,Gross Adds,Prepaid Gross Adds,2025-03-21 08:01:02,20250301,20250320,"1,242,357","604,369","637,988"
5,PREPAID,Gross Adds,Prepaid Gross Adds - GEO Channel,2025-03-21 08:01:03,20250301,20250320,"1,226,657","602,462","624,195"
6,PREPAID,Inflow M1,Prepaid Inflow M1,2025-03-21 08:01:03,20250301,20250319,"250,580,326","114,874,075","135,706,252"
7,PREPAID,Inflow M1,Prepaid Inflow M1 - GEO Channel,2025-03-21 08:01:03,20250301,20250319,"245,712,696","113,777,153","131,935,543"
8,PREPAID,Inflow M2,Prepaid Inflow M2,2025-03-21 08:01:03,20250301,20250319,"78,253,998","36,843,247","41,410,752"
9,PREPAID,Inflow M2,Prepaid Inflow M2 - GEO Channel,2025-03-21 08:01:03,20250301,20250319,"77,356,394","36,788,195","40,568,199"


In [6]:
''' POSTPAID Summary : 8 rows, 16 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'POSTPAID'
# channel_grp = 'GEO'

postpaid_df = chk_src_df.copy()
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==curr_mth]
postpaid_df = postpaid_df.loc[postpaid_df['PRODUCT']==product]
# postpaid_df = postpaid_df.loc[postpaid_df['CHANNEL_GRP']==channel_grp]

postpaid_df['ALL'] = postpaid_df['P']
postpaid_df['TRUE'] = np.where(postpaid_df['COMP_CD']=='TRUE', postpaid_df['P'], 0)
postpaid_df['DTAC'] = np.where(postpaid_df['COMP_CD']=='DTAC', postpaid_df['P'], 0)
postpaid_df['TMP_NAME'] = postpaid_df['METRIC_NAME'].replace(r' : DTAC| : TMH', '', regex=True)

postpaid_df = postpaid_df.groupby(['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum'})
postpaid_df = postpaid_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'TMP_NAME']).reset_index()

mod_col_list = postpaid_df.iloc[:, 6:9].columns.tolist()
for col in mod_col_list:
    postpaid_df[col] = postpaid_df[col].apply(lambda x: format(x, ',.0f'))

postpaid_df

,PRODUCT,METRIC_GRP,TMP_NAME,LOAD_DATE,START_DAY,END_DAY,ALL,TRUE,DTAC
0,POSTPAID,Gross Adds,Postpaid Gross Adds B2B,2025-03-21 08:21:54,20250301,20250320,"22,826","15,578","7,248"
1,POSTPAID,Gross Adds,Postpaid Gross Adds B2B - GEO Channel,2025-03-21 08:21:54,20250301,20250320,"2,455","2,028",427
2,POSTPAID,Gross Adds,Postpaid Gross Adds B2C,2025-03-21 08:21:54,20250301,20250320,"130,396","105,224","25,172"
3,POSTPAID,Gross Adds,Postpaid Gross Adds B2C - GEO Channel,2025-03-21 08:21:54,20250301,20250320,"64,227","54,969","9,258"
4,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B,2025-03-21 08:21:54,20250301,20250320,"11,236,931","3,621,370","7,615,562"
5,POSTPAID,Inflow M1,Postpaid Inflow M1 B2B - GEO Channel,2025-03-21 08:21:54,20250301,20250320,"2,191,874","1,940,070","251,804"
6,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C,2025-03-21 08:21:54,20250301,20250320,"58,443,884","47,842,798","10,601,086"
7,POSTPAID,Inflow M1,Postpaid Inflow M1 B2C - GEO Channel,2025-03-21 08:21:54,20250301,20250320,"31,709,324","26,595,523","5,113,801"


In [7]:
''' TOL Summary : 20 KPIs'''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TOL'
# channel_grp = 'GEO'

tol_df = chk_src_df.copy()
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==curr_mth]
tol_df = tol_df.loc[tol_df['PRODUCT']==product]
# tol_df = tol_df.loc[tol_df['CHANNEL_GRP']==channel_grp]

tol_df = tol_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tol_df = tol_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tol_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tol_df[col] = tol_df[col].apply(lambda x: format(x, ',.0f'))

tol_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TOL,Gross Adds,TOL Gross Adds Connected : Consumer,2025-03-21 08:00:09,20250301,20250320,"34,389"
1,TOL,Gross Adds,TOL Gross Adds Connected : Consumer (Install L...,2025-03-21 08:00:09,20250301,20250320,"34,389"
2,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Channel,2025-03-21 08:00:09,20250301,20250320,"27,098"
3,TOL,Gross Adds,TOL Gross Adds Connected : Consumer - GEO Chan...,2025-03-21 08:00:09,20250301,20250320,"27,098"
4,TOL,Gross Adds,TOL Gross Adds Connected : DataService,2025-03-21 08:00:09,20250301,20250320,"1,009"
5,TOL,Gross Adds,TOL Gross Adds Connected : DataService (Instal...,2025-03-21 08:00:09,20250301,20250320,"1,009"
6,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-03-21 08:00:09,20250304,20250318,23
7,TOL,Gross Adds,TOL Gross Adds Connected : DataService - GEO C...,2025-03-21 08:00:09,20250304,20250318,23
8,TOL,Gross Adds,TOL Gross Adds Registration : Consumer,2025-03-21 08:00:09,20250301,20250320,"48,137"
9,TOL,Gross Adds,TOL Gross Adds Registration : Consumer - GEO C...,2025-03-21 08:00:09,20250301,20250320,"37,209"


In [8]:
''' TVS Summary : 8 KPIs '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
product = 'TVS'
# channel_grp = 'GEO'

tvs_df = chk_src_df.copy()
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==curr_mth]
tvs_df = tvs_df.loc[tvs_df['PRODUCT']==product]
# tvs_df = tvs_df.loc[tvs_df['CHANNEL_GRP']==channel_grp]

tvs_df = tvs_df.groupby(['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum'})
tvs_df = tvs_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_GRP', 'METRIC_NAME']).reset_index()

mod_col_list = tvs_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    tvs_df[col] = tvs_df[col].apply(lambda x: format(x, ',.0f'))

tvs_df

,PRODUCT,METRIC_GRP,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P
0,TVS,Gross Adds,TVS CMDU Gross Adds,2025-03-21 08:28:07,20250301,20250318,32
1,TVS,Gross Adds,TVS CMDU Gross Adds - GEO Channel,2025-03-21 08:28:07,20250304,20250318,6
2,TVS,Gross Adds,TVS Now Gross Adds,2025-03-21 08:28:07,20250301,20250319,"1,541"
3,TVS,Gross Adds,TVS Now Gross Adds - GEO Channel,2025-03-21 08:28:07,20250301,20250319,"1,540"
4,TVS,Inflow M1,TVS CMDU Inflow M1,2025-03-21 08:28:07,20250301,20250318,"215,984"
5,TVS,Inflow M1,TVS CMDU Inflow M1 - GEO Channel,2025-03-21 08:28:07,20250304,20250318,"17,856"
6,TVS,Inflow M1,TVS Now Inflow M1,2025-03-21 08:28:07,20250301,20250319,"365,994"
7,TVS,Inflow M1,TVS Now Inflow M1 - GEO Channel,2025-03-21 08:28:07,20250301,20250319,"365,883"


### Monitoring

In [9]:
''' Current Month Gross Adds '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH']==curr_mth]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-03-21 08:21:54,20250301,20250319,0,"25,172","24,966","24,966","24,966","24,966","24,966",7,4898,24004
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-03-21 08:21:54,20250301,20250319,0,"7,248","2,812","2,812","2,812","2,812","2,812",7,950,3183
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-03-21 08:21:54,20250301,20250320,0,"105,224","74,777","74,777","74,777","74,777","74,777",7,1392,18893
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-03-21 08:21:54,20250301,20250320,0,"15,578","2,033","2,033","2,033","2,033","2,033",7,674,3464
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-03-21 08:01:02,20250301,20250319,0,"637,988","553,264","553,264","605,828","617,814","614,458",7,6560,65890
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-03-21 08:01:02,20250301,20250320,0,"604,369","458,069","458,069","443,005","458,069","458,043",7,5685,72152
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-03-21 08:00:09,20250301,20250320,0,"34,389","27,100","27,100","26,876","27,100","27,100",7,1148,15194
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-03-21 08:00:09,20250301,20250320,0,"1,009",23,23,23,23,23,7,58,92
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-03-21 08:00:09,20250301,20250320,0,"48,137","37,206","37,206","36,930","37,206","37,206",7,1154,15725
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-03-21 08:00:09,20250301,20250320,0,"1,063",38,38,38,38,38,7,49,114


In [10]:
''' Average Gross Adds per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
ga_metric_list = [
    'DB2S010100CS' #Postpaid Gross Adds B2C : DTAC
    , 'DB2S020100CS' #Postpaid Gross Adds B2B : DTAC
    , 'TB2S010100CS' #Postpaid Gross Adds B2C : TMH
    , 'TB2S020100CS' #Postpaid Gross Adds B2B : TMH
    , 'DB1S000101CS' #Prepaid Gross Adds : DTAC
    , 'TB1S000101CS' #Prepaid Gross Adds : TMH
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB3S000103CS' #TOL Gross Adds Connected : DataService
    , 'TB3S000104CS' #TOL Gross Adds Registration : Consumer
    , 'TB3S000105CS' #TOL Gross Adds Registration : DataService
    , 'TB4S001004CS' #TVS CMDU Gross Adds
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

src_ga_overall_df = chk_src_df.copy()
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_ga_overall_df = src_ga_overall_df.loc[src_ga_overall_df['METRIC_CD'].isin(ga_metric_list)]
src_ga_overall_df = src_ga_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_ga_overall_df = src_ga_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_ga_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_ga_overall_df[col] = src_ga_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_ga_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,2025-03-21 08:21:54,20240101,20250228,0,"62,952","61,001","61,001","61,001","61,001","61,001",7,6204,657186
1,POSTPAID,DB2S020100CS,Postpaid Gross Adds B2B : DTAC,2025-03-21 08:21:54,20240101,20250228,0,"9,947","5,395","5,395","5,395","5,395","5,395",7,1528,74902
2,POSTPAID,TB2S010100CS,Postpaid Gross Adds B2C : TMH,2025-03-21 08:21:54,20240101,20250228,0,"152,413","102,608","102,608","102,608","102,608","102,608",7,2211,409848
3,POSTPAID,TB2S020100CS,Postpaid Gross Adds B2B : TMH,2025-03-21 08:21:54,20240101,20250228,0,"24,861","8,465","8,465","8,465","8,465","8,465",7,781,59643
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,2025-03-21 08:01:02,20240101,20250228,0,"1,370,010","1,323,411","1,323,411","1,305,589","1,323,624","1,315,535",7,7441,1577135
5,PREPAID,TB1S000101CS,Prepaid Gross Adds : TMH,2025-03-21 08:01:02,20240101,20250228,0,"1,149,759","954,285","954,285","932,410","954,285","954,219",7,5817,1671756
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,2025-03-21 08:00:09,20240102,20250228,0,"46,522","32,998","32,998","34,286","34,612","34,597",7,1214,307979
7,TOL,TB3S000103CS,TOL Gross Adds Connected : DataService,2025-03-21 08:00:09,20240102,20250228,0,"1,574",34,34,35,35,35,7,99,2005
8,TOL,TB3S000104CS,TOL Gross Adds Registration : Consumer,2025-03-21 08:00:09,20240101,20250228,0,"63,752","44,821","44,821","46,597","47,021","46,996",7,1243,324801
9,TOL,TB3S000105CS,TOL Gross Adds Registration : DataService,2025-03-21 08:00:09,20240101,20250228,0,"3,488","1,103","1,103","1,094","1,114","1,114",7,620,25550


In [11]:
''' Current Month Inflow '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH']==curr_mth]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-03-21 08:21:54,20250301,20250319,0,"10,601,086","10,482,235","10,482,235","10,482,235","10,482,235","10,482,235",7,4898,24004
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-03-21 08:21:54,20250301,20250319,0,"7,615,562","1,033,326","1,033,326","1,033,326","1,033,326","1,033,326",7,950,3183
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-03-21 08:21:54,20250301,20250320,0,"47,842,798","38,284,958","38,284,958","38,284,958","38,284,958","38,284,958",7,1392,18893
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-03-21 08:21:54,20250301,20250320,0,"3,621,370","1,942,965","1,942,965","1,942,965","1,942,965","1,942,965",7,674,3464
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-03-21 08:01:03,20250301,20250319,0,"135,706,252","120,205,309","120,205,309","130,258,203","133,081,322","132,350,045",7,6546,91404
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-03-21 08:01:03,20250301,20250319,0,"41,410,752","35,298,266","35,298,266","40,318,183","40,906,534","40,473,867",7,6723,115394
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-03-21 08:01:03,20250301,20250318,0,"114,874,075","88,284,886","88,284,886","85,280,948","88,284,886","88,280,667",7,5626,78775
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-03-21 08:01:03,20250301,20250318,0,"36,843,247","36,149,868","36,149,868","35,435,018","36,149,868","36,145,416",7,5631,89133
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-03-21 08:00:09,20250301,20250320,0,"18,873,871","15,137,516","15,137,516","14,999,835","15,137,516","15,137,516",7,1148,15194
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-03-21 08:00:09,20250301,20250320,0,"1,306,969","58,878","58,878","58,878","58,878","58,878",7,58,92


In [12]:
''' Average Inflow per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
inflow_metric_list = [
    'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
    , 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
    , 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
    , 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH
    , 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
    , 'TB1R000900CS' #Prepaid Inflow M1 : TMH
    , 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
    , 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
    , 'TB4R001004CS' #TVS CMDU Inflow M1
    , 'TB4R001700CS' #TVS Now Inflow M1
    , 'DB1R001000CS' #Prepaid Inflow M2 : DTAC
    , 'TB1R001000CS' #Prepaid Inflow M2 : TMH
    ]

src_inflow_overall_df = chk_src_df.copy()
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_inflow_overall_df = src_inflow_overall_df.loc[src_inflow_overall_df['METRIC_CD'].isin(inflow_metric_list)]
src_inflow_overall_df = src_inflow_overall_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_inflow_overall_df = src_inflow_overall_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

mod_col_list = src_inflow_overall_df.iloc[:, 6:13].columns.tolist()
for col in mod_col_list:
    src_inflow_overall_df[col] = src_inflow_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_inflow_overall_df

,PRODUCT,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,POSTPAID,DB2R010500CS,Postpaid Inflow M1 B2C : DTAC,2025-03-21 08:21:54,20240101,20250228,0,"381,577,081","372,746,838","372,746,838","372,746,838","372,746,838","372,746,838",7,6204,657186
1,POSTPAID,DB2R020500CS,Postpaid Inflow M1 B2B : DTAC,2025-03-21 08:21:54,20240101,20250228,0,"137,554,957","25,803,178","25,803,178","25,803,178","25,803,178","25,803,178",7,1528,74902
2,POSTPAID,TB2R010500CS,Postpaid Inflow M1 B2C : TMH,2025-03-21 08:21:54,20240101,20250228,0,"892,798,861","686,190,728","686,190,728","686,190,728","686,190,728","686,190,728",7,2211,409848
3,POSTPAID,TB2R020500CS,Postpaid Inflow M1 B2B : TMH,2025-03-21 08:21:54,20240101,20250228,0,"78,786,203","33,347,478","33,347,478","33,347,478","33,347,478","33,347,478",7,781,59643
4,PREPAID,DB1R000900CS,Prepaid Inflow M1 : DTAC,2025-03-21 08:01:03,20240101,20250228,0,"3,575,402,630","3,516,813,717","3,516,813,717","3,462,937,926","3,517,489,107","3,493,902,722",7,7418,2373491
5,PREPAID,DB1R001000CS,Prepaid Inflow M2 : DTAC,2025-03-21 08:01:03,20240201,20250228,0,"1,018,625,947","1,005,306,086","1,005,306,086","993,879,464","1,005,497,791","993,984,000",7,7294,2559496
6,PREPAID,TB1R000900CS,Prepaid Inflow M1 : TMH,2025-03-21 08:01:03,20240101,20250228,0,"2,736,943,469","2,376,984,888","2,376,984,888","2,308,755,337","2,376,984,888","2,376,829,121",7,5793,1952685
7,PREPAID,TB1R001000CS,Prepaid Inflow M2 : TMH,2025-03-21 08:01:03,20240101,20250228,0,"997,560,166","988,072,190","988,072,190","970,164,151","988,072,190","987,964,726",7,5739,2116490
8,TOL,TB3R000601CS,TOL Inflow M1 Connected : Consumer,2025-03-21 08:00:09,20240102,20250228,0,"359,959,644","260,255,836","260,255,836","270,547,034","273,372,494","273,249,869",7,1214,307979
9,TOL,TB3R000602CS,TOL Inflow M1 Connected : DataService,2025-03-21 08:00:09,20240102,20250228,0,"26,851,654","1,478,429","1,478,429","1,514,815","1,517,914","1,517,914",7,99,2005


### All Metric

In [13]:
''' METRIC Summary '''

# src_t_sales_mth_df = chk_src_df.copy()
# src_t_sales_mth_df = src_t_sales_mth_df.groupby(['PRODUCT', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_sales_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=10)}')


     PRODUCT       METRIC_CD                                                              METRIC_NAME           LOAD_DATE  START_DAY   END_DAY              P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   POSTPAID    DB2R010500CG                              Postpaid Inflow M1 B2C : DTAC - GEO Channel 2025-03-21 08:21:54   20240101  20250319    152,953,107              7          798   215942
1   POSTPAID    DB2R010500CS                                            Postpaid Inflow M1 B2C : DTAC 2025-03-21 08:21:54   20240101  20250319    392,178,166              7         6204   681190
2   POSTPAID    DB2R020500CG                              Postpaid Inflow M1 B2B : DTAC - GEO Channel 2025-03-21 08:21:54   20240101  20250319      6,226,770              7          311    23816
3   POSTPAID    DB2R020500CS                                            Postpaid Inflow M1 B2B : DTAC 2025-03-21 08:21:54   20240101  20250319    145,170,519              7         1528    78085
4   POSTPAID    DB2S0101

In [14]:
''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_sales_mth_df_cols = ['PRODUCT', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_sales_mth_df = chk_src_df[src_t_sales_mth_df_cols].copy()
# src_t_sales_mth_df = src_t_sales_mth_df.loc[src_t_sales_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_sales_mth_df = src_t_sales_mth_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_sales_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_sales_mth_df[col] = src_t_sales_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_sales_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202503

     PRODUCT       METRIC_CD                                                              METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C            P            G            H           HH         CCAA       CCAATT
0   POSTPAID    DB2R010500CG                              Postpaid Inflow M1 B2C : DTAC - GEO Channel 2025-03-21 08:21:54     202503   20250301  20250319  0    5,113,801    5,109,210    5,109,210    5,109,210    5,109,210    5,109,210
1   POSTPAID    DB2R010500CS                                            Postpaid Inflow M1 B2C : DTAC 2025-03-21 08:21:54     202503   20250301  20250319  0   10,601,086   10,482,235   10,482,235   10,482,235   10,482,235   10,482,235
2   POSTPAID    DB2R020500CG                              Postpaid Inflow M1 B2B : DTAC - GEO Channel 2025-03-21 08:21:54     202503   20250301  20250319  0      251,804      251,305      251,305      251,305      251,305      251,305
3   POSTPAI

## Actual : Transaction
->  DIM_CORP_KPI

In [15]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN METRIC_VALUE END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
    FROM CDSAPPO.DIM_CORP_KPI
    WHERE REGEXP_LIKE(METRIC_CD, 'CS$|CG$') -->> 2025 KPIs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PRODUCT, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-03-21, 10:37:20



   -> Daily DataFrame : 5355 rows, 12 columns

TDMDBPR : Disconnected


In [16]:
''' METRIC by Day '''

v_tm_key_day = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

src_sale_day_df = chk_src_df.copy()
src_sale_day_df = src_sale_day_df.loc[src_sale_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_sale_day_df = src_sale_day_df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_sale_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_sale_day_df[col] = src_sale_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_sale_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_sale_day_df.to_string(max_cols=12)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_sale_day_df...

   -> DataFrame : 5355 rows, 12 columns


In [17]:
''' xB1S000101CS : Prepaid Gross Adds : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB1S000101CS', 'TB1S000101CS']
print(f'\nDB1S000101CS : Prepaid Gross Adds : DTAC\nTB1S000101CS : Prepaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB1S000101CS_day_df = chk_src_df.copy()
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB1S000101CS_day_df = xB1S000101CS_day_df.loc[xB1S000101CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB1S000101CS_day_df = xB1S000101CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB1S000101CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB1S000101CS_day_df[col] = xB1S000101CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB1S000101CS_day_df


DB1S000101CS : Prepaid Gross Adds : DTAC
TB1S000101CS : Prepaid Gross Adds : TMH

   -> v_metric_cd: ['DB1S000101CS', 'TB1S000101CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250301,2025-03-21 08:01:02,0,"39,049","33,585","33,585","36,734","37,288","37,073"
1,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250302,2025-03-21 08:01:02,0,"41,058","33,971","33,971","37,310","37,911","37,496"
2,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250303,2025-03-21 08:01:02,0,"35,155","30,338","30,338","33,573","34,442","34,283"
3,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250304,2025-03-21 08:01:02,0,"33,235","28,889","28,889","32,039","32,829","32,692"
4,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250305,2025-03-21 08:01:02,0,"34,208","29,931","29,931","33,070","33,765","33,632"
5,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250306,2025-03-21 08:01:02,0,"36,486","32,198","32,198","35,114","36,019","35,856"
6,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250307,2025-03-21 08:01:02,0,"38,198","32,972","32,972","35,636","36,527","36,388"
7,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250308,2025-03-21 08:01:02,0,"33,204","29,014","29,014","31,950","32,551","32,396"
8,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250309,2025-03-21 08:01:02,0,"33,337","29,011","29,011","32,019","32,537","32,180"
9,PREPAID,DB1S000101CS,Prepaid Gross Adds : DTAC,20250310,2025-03-21 08:01:02,0,"32,484","28,386","28,386","31,175","31,791","31,653"


In [18]:
''' xB2S010100CS : Postpaid Gross Adds B2C : xxx '''

v_tm_key_day = 20250301
v_metric_cd = ['DB2S010100CS', 'TB2S010100CS']
print(f'\nDB2S010100CS : Postpaid Gross Adds : DTAC\nTB2S010100CS : Postpaid Gross Adds : TMH\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

xB2S010100CS_day_df = chk_src_df.copy()
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
xB2S010100CS_day_df = xB2S010100CS_day_df.loc[xB2S010100CS_day_df['METRIC_CD'].isin(v_metric_cd)]
xB2S010100CS_day_df = xB2S010100CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = xB2S010100CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    xB2S010100CS_day_df[col] = xB2S010100CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
xB2S010100CS_day_df


DB2S010100CS : Postpaid Gross Adds : DTAC
TB2S010100CS : Postpaid Gross Adds : TMH

   -> v_metric_cd: ['DB2S010100CS', 'TB2S010100CS']
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250301,2025-03-21 08:21:54,0,"1,143","1,138","1,138","1,138","1,138","1,138"
1,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250302,2025-03-21 08:21:54,0,888,887,887,887,887,887
2,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250303,2025-03-21 08:21:54,0,"1,362","1,354","1,354","1,354","1,354","1,354"
3,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250304,2025-03-21 08:21:54,0,"1,696","1,684","1,684","1,684","1,684","1,684"
4,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250305,2025-03-21 08:21:54,0,"1,383","1,373","1,373","1,373","1,373","1,373"
5,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250306,2025-03-21 08:21:54,0,"1,707","1,688","1,688","1,688","1,688","1,688"
6,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250307,2025-03-21 08:21:54,0,"1,366","1,359","1,359","1,359","1,359","1,359"
7,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250308,2025-03-21 08:21:54,0,"1,164","1,156","1,156","1,156","1,156","1,156"
8,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250309,2025-03-21 08:21:54,0,834,832,832,832,832,832
9,POSTPAID,DB2S010100CS,Postpaid Gross Adds B2C : DTAC,20250310,2025-03-21 08:21:54,0,"1,525","1,514","1,514","1,514","1,514","1,514"


In [19]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_day = 20250301
v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3S000102CS_day_df = chk_src_df.copy()
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3S000102CS_day_df = TB3S000102CS_day_df.loc[TB3S000102CS_day_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_day_df = TB3S000102CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3S000102CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_day_df[col] = TB3S000102CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000102CS_day_df


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_metric_cd: TB3S000102CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250301,2025-03-21 08:00:09,0,"1,759","1,376","1,376","1,365","1,376","1,376"
1,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250302,2025-03-21 08:00:09,0,"1,603","1,211","1,211","1,193","1,211","1,211"
2,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250303,2025-03-21 08:00:09,0,"1,737","1,392","1,392","1,373","1,392","1,392"
3,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250304,2025-03-21 08:00:09,0,"1,789","1,449","1,449","1,441","1,449","1,449"
4,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250305,2025-03-21 08:00:09,0,"1,733","1,417","1,417","1,402","1,417","1,417"
5,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250306,2025-03-21 08:00:09,0,"1,726","1,408","1,408","1,400","1,408","1,408"
6,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250307,2025-03-21 08:00:09,0,"1,657","1,348","1,348","1,338","1,348","1,348"
7,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250308,2025-03-21 08:00:09,0,"1,806","1,334","1,334","1,323","1,334","1,334"
8,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250309,2025-03-21 08:00:09,0,"1,711","1,265","1,265","1,259","1,265","1,265"
9,TOL,TB3S000102CS,TOL Gross Adds Connected : Consumer,20250310,2025-03-21 08:00:09,0,"1,826","1,418","1,418","1,410","1,418","1,418"


In [20]:
''' TB4S001400CS : TVS Now Gross Adds '''

v_tm_key_day = 20250301
v_metric_cd = 'TB4S001400CS'
print(f'\nTB4S001400CS : TVS Now Gross Adds\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4S001400CS_day_df = chk_src_df.copy()
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4S001400CS_day_df = TB4S001400CS_day_df.loc[TB4S001400CS_day_df['METRIC_CD']==v_metric_cd]
TB4S001400CS_day_df = TB4S001400CS_day_df.fillna(0).sort_values(by=['PRODUCT', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4S001400CS_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4S001400CS_day_df[col] = TB4S001400CS_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4S001400CS_day_df


TB4S001400CS : TVS Now Gross Adds

   -> v_metric_cd: TB4S001400CS
   -> v_tm_key_day: 20250301


,PRODUCT,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,TVS,TB4S001400CS,TVS Now Gross Adds,20250301,2025-03-21 08:28:07,0,24,24,24,24,24,24
1,TVS,TB4S001400CS,TVS Now Gross Adds,20250302,2025-03-21 08:28:07,0,47,47,47,47,47,47
2,TVS,TB4S001400CS,TVS Now Gross Adds,20250303,2025-03-21 08:28:07,0,22,22,22,22,22,22
3,TVS,TB4S001400CS,TVS Now Gross Adds,20250304,2025-03-21 08:28:07,0,23,23,23,23,23,23
4,TVS,TB4S001400CS,TVS Now Gross Adds,20250305,2025-03-21 08:28:07,0,79,79,79,79,79,79
5,TVS,TB4S001400CS,TVS Now Gross Adds,20250306,2025-03-21 08:28:07,0,214,214,214,214,214,214
6,TVS,TB4S001400CS,TVS Now Gross Adds,20250307,2025-03-21 08:28:07,0,162,162,162,162,162,162
7,TVS,TB4S001400CS,TVS Now Gross Adds,20250308,2025-03-21 08:28:07,0,143,143,143,143,143,143
8,TVS,TB4S001400CS,TVS Now Gross Adds,20250309,2025-03-21 08:28:07,0,145,145,145,145,145,145
9,TVS,TB4S001400CS,TVS Now Gross Adds,20250310,2025-03-21 08:28:07,0,87,87,87,87,87,87


## Actual : Issue
->  DIM_CORP_KPI

In [21]:
''' Execute Issue by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250101
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ T.TM_KEY_DAY AS TM_KEY_MTH
        , T.METRIC_CD, T.METRIC_NAME, T.AREA_TYPE, T.AREA_CD, T.AREA_DESC
        , T.METRIC_VALUE AS TARGET
        , A.ACTUAL
        , O.ORGID_P, O.ZONE_TYPE, O.ORGID_G, O.TDS_SGMD, O.ORGID_H, O.HOP_HINT, O.ORGID_HH, O.D_CLUSTER
        
    FROM CDSAPPO.DIM_CORP_TARGET T

    LEFT JOIN (
        SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
            , PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
            , SUM(METRIC_VALUE) AS ACTUAL, MAX(LOAD_DATE) LOAD_DATE
        FROM CDSAPPO.DIM_CORP_KPI
        WHERE METRIC_CD = 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        AND AREA_TYPE = 'CCAA'
        AND TM_KEY_DAY >= 20250101
        GROUP BY SUBSTR(TM_KEY_DAY,1,6), PRODUCT, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC
    ) A
        ON A.METRIC_CD = T.METRIC_CD
        AND A.AREA_CD = T.AREA_CD
        AND A.TM_KEY_MTH = T.TM_KEY_DAY
        
    LEFT JOIN (
        SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG 
            , SUBSTR(CCAATT,1,4) AS CCAA, DISTRICT_EN
        FROM CDSAPPO.DIM_MOOC_AREA NOLOCK
        WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
    ) O
        ON O.CCAA = T.AREA_CD

    WHERE T.METRIC_CD = 'TB3S000102CS'
    AND T.AREA_TYPE = 'CCAA'
    AND T.TM_KEY_DAY >= 202501

    --ORDER BY T.TM_KEY_DAY, T.METRIC_CD, T.AREA_TYPE, O.ZONE_TYPE, O.ORGID_G, O.ORGID_H, O.ORGID_HH
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-03-21, 10:38:23



   -> Issue DataFrame : 2399 rows, 16 columns

TDMDBPR : Disconnected


In [22]:
''' TB3S000102CS : TOL Gross Adds Connected : Consumer '''

v_tm_key_mth = 202501
# v_metric_cd = 'TB3S000102CS'
print(f'\nTB3S000102CS : TOL Gross Adds Connected : Consumer\n')
# print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

TB3S000102CS_issue_df = chk_src_df.copy()
TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['TM_KEY_MTH']==v_tm_key_mth]
TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'] > 0]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['ACTUAL'].isna()]
# TB3S000102CS_issue_df = TB3S000102CS_issue_df.loc[TB3S000102CS_issue_df['METRIC_CD']==v_metric_cd]
TB3S000102CS_issue_df = TB3S000102CS_issue_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB3S000102CS_issue_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB3S000102CS_issue_df[col] = TB3S000102CS_issue_df[col].apply(lambda x: format(x, ',.0f'))
    
# TB3S000102CS_issue_df
print(f'\n{TB3S000102CS_issue_df.to_string(max_cols=100)}')


TB3S000102CS : TOL Gross Adds Connected : Consumer

   -> v_tm_key_mth: 202501

     TM_KEY_MTH     METRIC_CD                          METRIC_NAME AREA_TYPE AREA_CD                                          AREA_DESC TARGET ACTUAL ORGID_P ZONE_TYPE ORGID_G                                          TDS_SGMD ORGID_H                                                                                                       HOP_HINT ORGID_HH                                                                                                  D_CLUSTER
0        202501  TB3S000102CS  TOL Gross Adds Connected : Consumer      CCAA    8102                                  Krabi_Khao Phanom     24     96       P       UPC     GX8                     Regional Management 8 (South)     81X                                                                                                KRABI, PHANGNGA   906100                                                                                                      KRA

## Output File
->  DIM_CORP_KPI

In [25]:
''' Execute Sales Monthly '''


# # Input parameter
# curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20240101
# print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# # Connect : TDMDBPR
# src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
# src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
# src_cur = src_conn.cursor()
# query = (f"""
#     SELECT /*+PARALLEL(8)*/ 
#         SUBSTR(TM_KEY_DAY,1,4) AS TM_KEY_YR, SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
#         , CASE WHEN REGEXP_LIKE(METRIC_CD, 'CS$|CG$') THEN 'Y2025' ELSE 'Y2024' END VER_FLAG
#         , PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#         , SUM(CASE WHEN AREA_TYPE = 'P' THEN METRIC_VALUE END) P_ACTUAL
#         , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) G
#         , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) H
#         , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) HH
#         , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) CCAA
#         , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN METRIC_VALUE END) CCAATT
#         , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) "016"
#         , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) "040"
#         , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) "080"
#         , MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY, MAX(LOAD_DATE) LOAD_DATE
#     FROM CDSAPPO.DIM_CORP_KPI
#     WHERE METRIC_CD NOT LIKE 'Error%'
#     AND EXISTS (
#         SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
#         FROM (
#             SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#             UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
#         ) TMP
#         WHERE TMP_CD = AREA_CD )
#     AND TM_KEY_DAY >= {v_start_date}
#     GROUP BY SUBSTR(TM_KEY_DAY,1,4), SUBSTR(TM_KEY_DAY,1,6), PRODUCT, COMP_CD, METRIC_CD, METRIC_NAME, REMARK, UPD_TM, PPN_TM
#     --ORDER BY 2, 3, PRODUCT, COMP_CD, METRIC_CD 
# """)


# try:
#     execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
#     print(f'\n   -> Execute query... {execute_datetime}')

#     # Get : Sales Monthly DataFrame
#     src_cur.execute(query)
#     rows = src_cur.fetchall()
#     monthly_sales_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
#     print(f'\n   -> Sales Monthly DataFrame : {monthly_sales_df.shape[0]} rows, {monthly_sales_df.shape[1]} columns')

#     src_cur.close()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     src_conn.close()
#     print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-21, 11:11:24

   -> Sales Monthly DataFrame : 1054 rows, 22 columns

TDMDBPR : Disconnected


In [26]:
''' Generate CSV file '''

# op_dir = 'data'
# op_monthly_file = f'sales_monthly_{str_curr_dt}'

# monthly_sales_df.to_csv(f'{op_dir}/tmp_{op_monthly_file}.csv', index=False, encoding='utf-8')
# print(f'\n   -> Generate "tmp_{op_monthly_file}.csv" successfully')

# monthly_sales_df.tail(3)


   -> Generate "tmp_sales_monthly_20250321.csv" successfully


,TM_KEY_YR,TM_KEY_MTH,VER_FLAG,PRODUCT,COMP_CD,METRIC_CD,METRIC_NAME,REMARK,UPD_TM,PPN_TM,...,H,HH,CCAA,CCAATT,016,040,080,START_DAY,END_DAY,LOAD_DATE
1051,2024,202403,Y2025,TVS,TRUE,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,KPI Metric 2025,20250304,20250304,...,213715.54,213715.54,213715.54,213715.54,None,None,NaN,20240301,20240330,2025-03-04 17:49:19
1052,2024,202408,Y2025,POSTPAID,DTAC,DB2S020100CG,Postpaid Gross Adds B2B : DTAC - GEO Channel,KPI Metric 2025,20250304,20250304,...,799.00,799.00,799.00,799.00,None,None,82.00,20240801,20240831,2025-03-04 17:48:58
1053,2024,202406,Y2025,TVS,TRUE,TB4R001004CG,TVS CMDU Inflow M1 - GEO Channel,KPI Metric 2025,20250304,20250304,...,223326.17,223326.17,223326.17,223326.17,None,None,6728.97,20240601,20240629,2025-03-04 17:49:19
